In [379]:
# Import libraries
import pandas as pd
import numpy as np
import re

# Import data downloaded from http://insideairbnb.com/
listings = pd.read_csv('/Users/kaileighstopa/Desktop/Python Practice/Hawaii Airbnb/Data/listings.csv')
reviews = pd.read_csv('/Users/kaileighstopa/Desktop/Python Practice/Hawaii Airbnb/Data/reviews.csv', encoding='ISO-8859-1')
hosts = pd.read_csv('/Users/kaileighstopa/Desktop/Python Practice/Hawaii Airbnb/Data/host.csv')

## Cleaning listings dataset

In [380]:
# Inspect dataset
display(listings.head())

# Length of dataset
display(len(listings))

# Inspect missing values 
display(listings.isna().sum())

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,3002.0,https://www.airbnb.com/rooms/3002,2.022120e+13,12/16/22,city scrape,"OCEAN VIEW 1BLK2BEACH,LANAI A/C,FULL KITCH,CEN...",Only renting while I'm gone on vacation or sta...,Waikiki has nice weather 90% percent if the ye...,https://a0.muscache.com/pictures/adf53d15-ab4b...,3364,...,4.87,4.97,4.74,92- 0251181,f,1,0,1,0,1.59
1,5269.0,https://www.airbnb.com/rooms/5269,2.022120e+13,12/15/22,city scrape,Upcountry Hospitality in the 'Auwai Suite,"The 'Auwai Suite is a lovely, self-contained a...","We are located on the ""sunny side"" of Waimea, ...",https://a0.muscache.com/pictures/5b52b72f-5a09...,7620,...,4.87,5.00,4.83,119-269-5808-01R,f,4,4,0,0,0.17
2,5387.0,https://www.airbnb.com/rooms/5387,2.022120e+13,12/15/22,city scrape,Hale Koa Studio & 1 Bedroom Units!!,This Wonderful Spacious Studio apt/flat is in ...,IN a Farm belt area with small commercial farm...,https://a0.muscache.com/pictures/1170713/dca6a...,7878,...,4.87,4.73,4.76,NaN,t,3,3,0,0,1.31
3,5389.0,https://www.airbnb.com/rooms/5389,2.022120e+13,12/15/22,city scrape,Keauhou Villa,It is less than 10 minute walk to the Keauhou ...,It is less than 10 minute walk to the Keauhou ...,https://a0.muscache.com/pictures/15520396/3b89...,7878,...,4.85,4.98,4.74,NaN,f,3,3,0,0,0.42
4,5390.0,https://www.airbnb.com/rooms/5390,2.022120e+13,12/16/22,city scrape,STAY AT PRINCE KUHIO!,"<b>The space</b><br />Prince Kuhio, Studio Uni...",NaN,https://a0.muscache.com/pictures/12955/af97ac5...,7887,...,4.78,4.89,4.64,"260050220000, TA-008-490-3424-01",f,1,1,0,0,1.03


30064

id                                                  0
listing_url                                         0
scrape_id                                           0
last_scraped                                        0
source                                              0
name                                                0
description                                       147
neighborhood_overview                           14097
picture_url                                         0
host_id                                             0
neighbourhood                                   14097
neighbourhood_cleansed                              0
neighbourhood_group_cleansed                        0
latitude                                            0
longitude                                           0
property_type                                       0
room_type                                           0
accommodates                                        0
bathrooms                   

There are quite a few feature with missing values in this dataset. Some features are missing values because there are listings that don't contain that feature. For example, not all hosts entered a listing description or neighborhood overview, which are freeform text entries. In addition, some listings don't have any reviews yet, leading to missing values in the review fields. Below, I will sort out which missing values make sense given the dataset, and which ones should be imputed. 

In [381]:
# Missing values in 'description', 'neighbourhood_overview' are fine as those fields are just not present in some listigs

# Check if there is anything useful in 'neighbourhood' column, or if we can drop

display(listings['neighbourhood'].unique())
display(listings['neighbourhood_cleansed'].unique())

# It looks like 'neighbourhood_cleansed' is a more conscise list of neighborhoods in Hawaii, while 'neighbourhood' 
# contains many unique values and is probably free-form text by the host. We will keep 'neighbourhood_cleansed' for analysis
listings = listings.drop(columns=['neighbourhood'])

# Rename 'neighbourhood_group_cleansed' to 'island' as it contains island information
listings.rename(columns={'neighbourhood_group_cleansed' : 'island'}, inplace=True)
listings.loc[listings['island']=='Honolulu', 'island'] = 'Oahu'

array(['Honolulu, Hawaii, United States',
       'Kamuela, Hawaii, United States',
       'Captain Cook, Hawaii, United States',
       'Kailua Kona, Hawaii, United States', nan,
       'Pāhoa, Hawaii, United States', 'Koloa, Hawaii, United States',
       'Kihei, Hawaii, United States',
       'Princeville, Hawaii, United States',
       'Kahuku, Hawaii, United States',
       'Kailua-Kona, Hawaii, United States',
       'Wailuku, Hawaii, United States', 'Haleiwa, Hawaii, United States',
       'Lahaina, Hawaii, United States',
       " Ma'alaea, Wailuku, Hawaii, United States",
       'Volcano, Hawaii, United States', 'Pahala, Hawaii, United States',
       'Haiku-Pauwela, Hawaii, United States',
       'Pepeekeo, Hawaii, United States', 'Paia, Hawaii, United States',
       'Makawao, Hawaii, United States', 'Kapolei, Hawaii, United States',
       'Kailua, Hawaii, United States', 'Keaau, Hawaii, United States',
       'Waianae, Hawaii, United States', 'Hanalei, Hawaii, United States

array(['Primary Urban Center', 'South Kohala', 'South Kona', 'North Kona',
       'Koloa-Poipu', 'Puna', 'Kihei-Makena', 'North Shore Kauai',
       'Koolauloa', 'Wailuku-Kahului', 'North Shore Oahu', 'Lahaina',
       'Kau', 'Paia-Haiku', 'South Hilo', 'Makawao-Pukalani-Kula',
       'Kapaa-Wailua', 'Ewa', 'Koolaupoko', 'Waianae', 'Waimea-Kekaha',
       'North Kohala', 'Lihue', 'East Honolulu', 'Molokai', 'North Hilo',
       'Hana', 'Hamakua', 'Lanai', 'Central Oahu'], dtype=object)

In [382]:
# 'bathrooms' column is entirely missing values, can go ahead and drop
listings = listings.drop(columns=['bathrooms'])

# Extract number of bathrooms in a listing from 'bathrooms_text' column 
listings['bathrooms_text'] = listings['bathrooms_text'].fillna('missing')
listings['bathroom_num'] = listings['bathrooms_text'].str.extract(r'(\d+(?:\.\d+)?)')
listings.loc[listings['bathrooms_text'].str.lower().str.contains('half-bath'), 'bathroom_num'] = 0.5

# Extract whether a bathroom is shared or not from 'bathrooms_text' column
listings['bathroom_shared'] = 0
listings.loc[listings['bathrooms_text'].str.lower().str.contains('shared'), 'bathroom_shared'] = 1

display(listings['bathroom_num'].isna().sum())

# There are still 21 missing values for 'bathroom_num', I will just impute with most common bathroom number which is 1 
listings['bathroom_num'] = listings['bathroom_num'].fillna(listings['bathroom_num'].mode()[0])
listings['bathroom_num'] = listings['bathroom_num'].astype(float)

display(listings[['bathroom_num', 'bathroom_shared']].describe())

21

,bathroom_num,bathroom_shared
count,30064.000000,30064.000000
mean,1.623603,0.027641
std,0.904354,0.163945
min,0.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,0.000000
75%,2.000000,0.000000
max,18.000000,1.000000


Now that the bathrooms features are fixed, I will take a look at the bedrooms feature. It's possible that any missing values for bedroom are because the listings are hotel rooms or studios, where there is no designated "bedroom". Let's see if this might be true.

In [383]:
# Missing values for bedrooms and beds
display(listings[listings['bedrooms'].isna()]['room_type'].value_counts())

Entire home/apt    3459
Private room        363
Hotel room            7
Name: room_type, dtype: int64

Unfortunately, it looks like the majority of missing bedroom values are for entire homes/apartments, which means that the 'bedrooms' information is probably just missing from the listing. There are a lot less missing values for # of beds ('beds'), so I will impute the missing values in 'bedrooms' using those values, assumimg that most bedrooms only have one bed in them. For listings that are missing both the 'beds' and 'bedrooms' feature, I will impute using the most common number of bedrooms across all listings.

In [384]:
# Impute missing 'bedrooms' values with 'beds' information, and vice versa
listings['bedrooms'] = listings['bedrooms'].fillna(listings['beds'])
listings['beds'] = listings['beds'].fillna(listings['bedrooms'])

display(listings[['bedrooms', 'beds']].isna().sum())

# There are still 87 missing values for 'bedrooms' and 'beds', I will impute these with the modes for both features
listings['bedrooms'] = listings['bedrooms'].fillna(listings['bedrooms'].mode()[0])
listings['beds'] = listings['beds'].fillna(listings['beds'].mode()[0])

bedrooms    87
beds        87
dtype: int64

In [385]:
# Extract $ sign from 'price' feature so it can be numeric
mask = listings['price'].notnull() & listings['price'].str.contains('$')
listings.loc[mask, 'price'] = pd.to_numeric(listings.loc[mask, 'price'].str.replace(',', '').str.lstrip('$'))

# Convert 'has_availability' and 'instant_bookable' to boolean
listings.loc[listings['has_availability']=='f', 'has_availability'] = False
listings.loc[listings['has_availability']=='t', 'has_availability'] = True

listings.loc[listings['instant_bookable']=='f', 'instant_bookable'] = False
listings.loc[listings['instant_bookable']=='t', 'instant_bookable'] = True

/var/folders/1d/wshdc1vs5sldbdlstdtwvqbr0000gn/T/ipykernel_28313/3938965379.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  listings.loc[mask, 'price'] = pd.to_numeric(listings.loc[mask, 'price'].str.replace(',', '').str.lstrip('$'))


The remaining missing values can remain in the dataset. The values 'description' and 'neighbourhood_overview' are missing from some listings because some hosts did not enter listing nad neighborhood descriptions. Review features, such as 'first_review', 'review_scores_rating', and 'reviews_per_month' are missing for listings with no reviews as of the time of data collection. I will not be using the 'license' feature in my analysis, so that is not relevant either. Below, I will drop 'calendar_updated', which contains only missing values, and other features that are not needed for my analysis going forward.

In [386]:
# Drop columns where all observations are missing and columns that are not relevant for analysis
listings = listings.drop(columns=['calendar_updated', 'scrape_id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'picture_url', 'latitude', 'longitude', 'minimum_minimum_nights',
                                  	'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm',
                                    'maximum_nights_avg_ntm', 'calendar_last_scraped', 'license'])

Next, I will take a look at descriptive statistics for the remaining features to make sure that there aren't any obvious outliers that need to be dealt with. 

In [387]:
# Inspect descriptive statistics
describe_listings = listings.describe()
display(describe_listings)

,id,host_id,accommodates,bedrooms,beds,price,minimum_nights,maximum_nights,availability_30,availability_60,...,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,bathroom_num,bathroom_shared
count,3.006400e+04,3.006400e+04,30064.000000,30064.000000,30064.000000,30064.000000,30064.000000,30064.000000,30064.000000,30064.000000,...,23240.000000,23238.000000,23237.000000,30064.000000,30064.000000,30064.000000,30064.000000,23290.000000,30064.000000,30064.000000
mean,1.937206e+17,1.450743e+08,4.738026,1.738159,2.538518,536.015521,7.852614,495.782597,7.729078,17.704464,...,4.821442,4.867748,4.678967,80.977382,67.186668,13.752495,0.012839,1.241942,1.623603,0.027641
std,3.073287e+17,1.452389e+08,2.464576,0.971893,1.536605,1528.681305,23.926141,451.984388,8.574517,18.017009,...,0.346405,0.246300,0.387124,125.440635,105.070240,75.054025,0.229617,1.364240,0.904354,0.163945
min,3.002000e+03,8.840000e+02,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000
25%,2.757391e+07,3.074786e+07,3.000000,1.000000,2.000000,178.000000,1.000000,60.000000,1.000000,3.000000,...,4.790000,4.830000,4.590000,2.000000,2.000000,0.000000,0.000000,0.300000,1.000000,0.000000
50%,4.741853e+07,9.700641e+07,4.000000,2.000000,2.000000,287.000000,2.000000,365.000000,5.000000,11.000000,...,4.940000,4.930000,4.770000,19.000000,14.000000,0.000000,0.000000,0.780000,1.000000,0.000000
75%,5.815162e+17,2.260004e+08,6.000000,2.000000,3.000000,480.000000,5.000000,1125.000000,11.000000,28.000000,...,5.000000,5.000000,4.910000,96.000000,82.000000,0.000000,0.000000,1.780000,2.000000,0.000000
max,7.815630e+17,4.913647e+08,16.000000,12.000000,32.000000,100000.000000,1000.000000,1825.000000,30.000000,60.000000,...,5.000000,5.000000,5.000000,570.000000,403.000000,534.000000,8.000000,41.800000,18.000000,1.000000


In [388]:
# Inspect rows where 'accommodates' == 0 
display(listings[listings['accommodates']==0])
# There is only row with a '0' value for 'accommodates' and it is a hotel room, so I will impute with 'accommodates' = 2, 
# as hotel rooms are most commonly for two people
listings.loc[listings['accommodates']==0, 'accommodates'] = 2

# Inspect rows where beds > 20, they could just be very large properties such as mansions
display(listings[listings['beds'] > 20][['name', 'description', 'bedrooms', 'bathroom_num']])
# Indeed they are mansions, manors, or estates, and the corresponding number of bedrooms matches up, 
# so I will leave them be

# The max value of 'minimum_nights' is 1000, which is a little surprising. The next highest value is 365, 
# which makes sense if someone is looking to rent their property for an entire year
display(listings[listings['minimum_nights']==1000])
# There is only one row, and it is a room in a boutique hotel, so a minimum stay of 1000 nights doesn't 
# make much sense. Impute 'minimum_nights' = 1, as most hotels do not require a minimum stay
listings.loc[listings['minimum_nights']==1000, 'minimum_nights'] = 1

,id,name,description,neighborhood_overview,host_id,neighbourhood_cleansed,island,property_type,room_type,accommodates,...,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,bathroom_num,bathroom_shared
13059,43309266.0,Castle Hilo Hawaiian Hotel,NaN,NaN,178505475,South Hilo,Hawaii,Room in hotel,Hotel room,0,...,4.78,4.26,False,1,0,0,0,10.3,1.0,0


,name,description,bedrooms,bathroom_num
7505,Fern Acres B&B 8 bed 8.5 bath Licensed.,"The quoted price is for the whole house, but w...",8.0,8.5
13203,Luxurious Hawaii Wedding Estate,"Welcome to the only luxury 10 bedroom, 5.5 on ...",10.0,5.5
15256,Hale Plumeria~oceanviewoahu~wVanS~Fast Net,This is one of the most unique homes near Waik...,7.0,6.0
15716,#1 Rated Luxury OCEAN FRONT Mansion! Sleeps 32,Welcome to Maui's most desirable vacation home...,11.0,11.0
20954,Orchid Isle Manor,Enjoy the sounds of nature when you stay at th...,8.0,7.5


,id,name,description,neighborhood_overview,host_id,neighbourhood_cleansed,island,property_type,room_type,accommodates,...,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,bathroom_num,bathroom_shared
7716,27994801.0,Romantic Ocean View Boutique hotel condo #2001,"Inn On the Park Condo, #2001 is a brand new ap...",It only take me 10 minutes to walk to the beac...,210552519,Primary Urban Center,Oahu,Room in boutique hotel,Private room,3,...,4.75,4.51,True,1,0,1,0,1.06,1.5,1


In [389]:
# Convert 'first_review' and 'last_review' to datetime 

listings['first_review'] = pd.to_datetime(listings['first_review'])
listings['last_review'] = pd.to_datetime(listings['last_review'])

In [390]:
# Export cleaned dataset to csv
listings.to_csv('/Users/kaileighstopa/Desktop/Python Practice/Hawaii Airbnb/Data/listing_cleaned.csv')

## Cleaning reviews dataset

In [391]:
# Inspect dataset
display(reviews.head())

# Length of dataset
display(len(reviews))

# Check for missing values
display(reviews.isna().sum())

# Check date range
reviews['date'] = pd.to_datetime(reviews['date'])
display('Review date range: ' + str(reviews['date'].dt.date.min()) + ' to ' +str(reviews['date'].dt.date.max()))


,listing_id,id,date,reviewer_id,reviewer_name,comments
0,3002.0,63686.0,7/11/10,155310,Debbie,"Lynne is very kind and accommodating, the loca..."
1,3002.0,71254.0,7/31/10,75939,Bonnie,I came back to the island on my own for my 25t...
2,3002.0,132520.0,11/2/10,189930,Alexandra,I had a wonderfull time in Hawaii. Lynne is a ...
3,3002.0,158315.0,12/25/10,291487,Shirley,"Lynn was a wonderful hostess and is a lovely, ..."
4,3002.0,168133.0,1/8/11,331679,Elizabeth,"Just a short walk to Waikiki beach, restaurant..."


961631

listing_id         0
id                 0
date               0
reviewer_id        0
reviewer_name      0
comments         195
dtype: int64

'Review date range: 2009-05-07 to 2022-12-15'

The dataset looks good already. There are some missing values for "comments", which is expected for reviewers who didn't leave any free-form text comments in their review.

Our dataset also contains reviews written in multiple languages in the "comments" feature, which will make it hard to do analysis later. I will use the langdetect library to create a column indicating the language that each review is written in. 

In [392]:
# Below code is commented out as it takes a long time to run, the processed dataset is re-imported in the next code block 

# from langdetect import detect 
# def detectlang(x):
#     try:
#         lang = detect(x)
#     except:
#         lang = 'Other'
#     return lang

# step_size = 10000 # print status update after processing this many comments
# num_comments = len(reviews)
# for i in range(0, num_comments, step_size):
#     reviews_subset = reviews.iloc[i:i+step_size]
#     reviews_subset['language'] = reviews_subset['comments'].apply(detectlang)
#     print(f"Processed {i+step_size} comments out of {num_comments}")

# reviews['language'] = reviews['comments'].apply(detectlang)

# # Export cleaned dataset to csv
# reviews.to_csv('/Users/kaileighstopa/Desktop/Python Practice/Hawaii Airbnb/Data/reviews_cleaned.csv')

In [393]:
# Import from cleaned file
reviews = pd.read_csv('/Users/kaileighstopa/Desktop/Python Practice/Hawaii Airbnb/Data/reviews_cleaned.csv', encoding='ISO-8859-1')
reviews.drop(columns='Unnamed: 0', inplace=True)
display(reviews.head())

,listing_id,id,date,reviewer_id,reviewer_name,comments,language
0,3002.0,63686.0,2010-07-11,155310,Debbie,"Lynne is very kind and accommodating, the loca...",en
1,3002.0,71254.0,2010-07-31,75939,Bonnie,I came back to the island on my own for my 25t...,en
2,3002.0,132520.0,2010-11-02,189930,Alexandra,I had a wonderfull time in Hawaii. Lynne is a ...,en
3,3002.0,158315.0,2010-12-25,291487,Shirley,"Lynn was a wonderful hostess and is a lovely, ...",en
4,3002.0,168133.0,2011-01-08,331679,Elizabeth,"Just a short walk to Waikiki beach, restaurant...",en


## Cleaning hosts dataset 

In [394]:
# Inspect dataset
display(hosts.head())

# Length of dataset
display(len(hosts))

# Check for missing values
display(hosts.isna().sum())

,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_verifications,host_has_profile_pic,host_identity_verified
0,884,https://www.airbnb.com/users/show/884,Doug,7/20/08,"Naples, FL",My wife Rebecca and I own a gorgeous ocean vie...,NaN,NaN,NaN,f,https://a0.muscache.com/im/pictures/user/35de0...,https://a0.muscache.com/im/pictures/user/35de0...,NaN,1.0,"['email', 'phone']",t,t
1,908,https://www.airbnb.com/users/show/908,Sherry,7/21/08,"Lahaina, HI",My name is Sherry Boonstra-Barbier and I have ...,within a few hours,100%,50%,f,https://a0.muscache.com/im/pictures/user/4fa8b...,https://a0.muscache.com/im/pictures/user/4fa8b...,Lahaina,18.0,"['email', 'phone', 'work_email']",t,t
2,2971,https://www.airbnb.com/users/show/2971,Donald,9/15/08,"Hakalau, HI",NaN,within an hour,100%,100%,f,https://a0.muscache.com/im/pictures/user/596bf...,https://a0.muscache.com/im/pictures/user/596bf...,NaN,1.0,"['email', 'phone']",t,t
3,3364,https://www.airbnb.com/users/show/3364,Lynne,9/30/08,"Honolulu, HI",Aloha! Alittle bit about me: I teach snorkelin...,within a day,100%,100%,f,https://a0.muscache.com/im/pictures/user/b6986...,https://a0.muscache.com/im/pictures/user/b6986...,Waikiki,1.0,"['email', 'phone']",t,t
4,4754,https://www.airbnb.com/users/show/4754,Christine,12/2/08,"Hilo, HI",We are seasoned travelers and the owners of Em...,NaN,NaN,NaN,f,https://a0.muscache.com/im/pictures/user/36817...,https://a0.muscache.com/im/pictures/user/36817...,Hilo,1.0,"['email', 'phone']",t,t


8308

host_id                      0
host_url                     0
host_name                    1
host_since                   1
host_location             1800
host_about                3678
host_response_time        1079
host_response_rate        1079
host_acceptance_rate       616
host_is_superhost           30
host_thumbnail_url           1
host_picture_url             1
host_neighbourhood        2885
host_listings_count          1
host_verifications           0
host_has_profile_pic         1
host_identity_verified       1
dtype: int64

In [395]:
# There are many features for which there is only 1 missing value - perhaps these are all due to the same observation
display(hosts[hosts['host_name'].isna()])
# Sure enough! Not sure what is going on with this observation, but I will drop it

# Impute 'host_is_superhost' as false for missing values
hosts['host_is_superhost'] = hosts['host_is_superhost'].fillna('f')

# Convert 'host_is_superhost', 'host_has_profile_pic' and 'host_identify_verified' to boolean
hosts.loc[hosts['host_is_superhost']=='f', 'host_is_superhost'] = False
hosts.loc[hosts['host_is_superhost']=='t', 'host_is_superhost'] = True

# Take out % symbols from 'host_response_rate' and 'host_acceptance_rate'
mask = hosts['host_response_rate'].notnull() & hosts['host_response_rate'].str.contains('%')
hosts.loc[mask, 'host_response_rate'] = pd.to_numeric(hosts.loc[mask, 'host_response_rate'].str.rstrip('%'))

mask = hosts['host_acceptance_rate'].notnull() & hosts['host_acceptance_rate'].str.contains('%')
hosts.loc[mask, 'host_acceptance_rate'] = pd.to_numeric(hosts.loc[mask, 'host_acceptance_rate'].str.rstrip('%'))

# Convert host_since to datetime 
hosts['host_since'] = pd.to_datetime(hosts['host_since'])

,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_verifications,host_has_profile_pic,host_identity_verified
413,2161416,https://www.airbnb.com/users/show/2161416,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,NaN,NaN,NaN,None,NaN,NaN


The remaining missing values are not concerning, as some host profiles are just missing the relevant information. For example, a host may not have an 'about' section or location on their profile. In addition, hosts that have not yet exchanged messages with guests or rented out their property may not have stats such as 'host_response_time' and 'host_acceptance_rate' available. 

In [396]:
# Drop features that are not relevant to my analysis 
hosts = hosts.drop(columns=['host_url', 'host_thumbnail_url', 'host_picture_url'])

In [397]:
# Look at summary statistics 
display(hosts.describe())

,host_id,host_listings_count
count,8.308000e+03,8307.000000
mean,1.451836e+08,8.023233
std,1.461807e+08,60.873949
min,8.840000e+02,1.000000
25%,2.498421e+07,1.000000
50%,9.083849e+07,1.000000
75%,2.273685e+08,3.000000
max,4.913647e+08,2577.000000


It looks like there might be hosts with multiple thousands of listings! Based on inspecting the host 'about' feature, it looks like the hosts with multiple hundreds of listings are mostly travel management companies that are operated through Airbnb, or large resorts with many units available. There's no straight forward way to check which of these values might be erroneous, so I will keep them for now.

In [398]:
# Display sample of 'about' feature for hosts with many listings
display(hosts[hosts['host_listings_count']>1000]['host_about'].values)

array(['Our selected accommodations combine the best features of a hotel and vacation rental home to ensure a wonderful and effortless stay! We take the guesswork out of where you are staying so you can concentrate on what you want to be doing.\r\n\r\nWe handpick the accommodations in the best properties all over the US offering great vacation rental experience and impeccable service: serviced apartments, oversized suites and accommodations in resorts with outstanding amenities and much more! We do this by testing them ourselves before the hit the list, so we know what to expect before you make the reservation. \r\n\r\nWe are available 24/7 to make sure all your special requests are taken care of and to assist with any questions or issues you might have throughout your stay!',
       'Going someplace new is always exciting. And what would that adventure be without a great place to lay your head at the end of the day? That is where I come in!\n\nAs an avid traveler myself, I know how th

In [399]:
# Export cleaned dataset to csv
hosts.to_csv('/Users/kaileighstopa/Desktop/Python Practice/Hawaii Airbnb/Data/hosts_cleaned.csv')